In [1]:
import time
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg') # 展示高清图，在 Jupyter Notebook 中设置 matplotlib 图形的输出格式为 SVG 格式

import torch
import torch.nn as nn
from scipy.signal import savgol_filter
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as TTS
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix, classification_report

import random #设置随机数，可以关闭
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
# 导入数据
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv(r".csv",encoding='utf-8') #encoding='GBK',防止中文乱码

X = data.iloc[:,1:-1]
y = data.iloc[:,-1]
X_SG = savgol_filter(X, 5, 2)
X_SG_mms = MinMaxScaler().fit_transform(X_SG)
Label = LabelEncoder().fit_transform(y)
data.iloc[:,1:-1] = X_SG_mms
data.iloc[:, -1] = Label
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]

X_dr = PCA(29).fit_transform(X)

# 转换为 torch 中的张量格式
X_tensor = torch.tensor(X_dr, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# 将数据移动到GPU
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)

Xtrain, Xtest, Ytrain, Ytest = TTS(X_dr,y,test_size=0.3,random_state=0)

# 转换为 torch 中的张量格式
X_train_tensor = torch.tensor(Xtrain, dtype=torch.float32)
X_test_tensor = torch.tensor(Xtest, dtype=torch.float32)
y_train_tensor = torch.tensor(Ytrain, dtype=torch.long)
y_test_array = Ytest.values
y_test_tensor = torch.tensor(y_test_array, dtype=torch.long)

# 将数据移动到GPU
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# 创建训练集和测试集的 TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# 创建 DataLoader 对象
# train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

train_size = len(train_dataset)
test_size = len(test_dataset)

train_size, test_size

FileNotFoundError: [Errno 2] No such file or directory: '.csv'

In [ ]:
# 搭建网络
class DNN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer_sizes, dropout_prob):
        ''' 搭建神经网络各层 '''
        super(DNN, self).__init__()

        self.hidden_layer_sizes = hidden_layer_sizes  # 调整隐藏层尺寸
        self.dropout_prob = dropout_prob  # 调整 dropout 概率

        layers = []
        prev_layer_size = input_size

        for layer_size in self.hidden_layer_sizes:
            layers.append(nn.Linear(prev_layer_size, layer_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(self.dropout_prob))
            prev_layer_size = layer_size

        layers.append(nn.Linear(prev_layer_size, num_classes))

        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        ''' 前向传播 '''
        y = self.net(x) # x 即输入数据
        return y # y 即输出数据

In [ ]:
# 优化超参数
num_classes = 33 #修改成你要识别的种类
input_size = X_tensor.shape[1]

hidden_layer_sizes_1 = 
dropout_prob_1 = 

In [ ]:
# 创建模型实例
model_1 = DNN(input_size, num_classes, hidden_layer_sizes_1, dropout_prob_1)
model_1.to(device)
print(model_1)

In [ ]:
# 训练网络——随机网格搜索——model_1
loss_fn = nn.CrossEntropyLoss(reduction='mean') # 对所有样本的损失求平均，得到一个标量
learning_rate_1 =  # 设置学习率
optimizer_1 = torch.optim.RMSprop(model_1.parameters(), lr=learning_rate_1)

epochs = 200
accuracies_1 = [] # 记录准确率变化的列表
losses_1 = [] # 记录损失函数变化的列表
model_1.train()
for epoch in range(epochs):
    total_correct_1 = 0
    total_samples_1 = 0
    batch_loss_1 = 0
    
    for (x, y) in train_dataloader: # 获取小批次的 x 与 y
        x, y = x.to(device), y.to(device)
        Pred = model_1(x) # 一次前向传播（小批量）
        _, Pred_classes = torch.max(Pred, dim=1)
        total_correct_1 += torch.sum(Pred_classes == y)
        total_samples_1 += y.size(0)
        loss = loss_fn(Pred, y) # 计算损失函数
        batch_loss_1 += loss.item()  # 累加当前 batch 的损失
        #losses.append(loss.item()) # 记录损失函数的变化
        optimizer_1.zero_grad() # 清理上一轮滞留的梯度
        loss.backward() # 一次反向传播
        optimizer_1.step() # 优化内部参数
        
    accuracy_1 = total_correct_1 / total_samples_1
    accuracies_1.append(accuracy_1.item())
    losses_1.append(batch_loss_1 / len(train_dataloader))  # 计算平均损失并保存

In [ ]:
# 测试网络——贝叶斯优化——model_2
correct_2 = 0
total_2 = 0
all_preds_2 = []
all_labels_2 = []
model_2.eval()
with torch.no_grad(): # 该局部关闭梯度计算功能
    for (x, y) in test_dataloader: # 获取小批次的 x 与 y
        x, y = x.to(device), y.to(device)
        Pred = model_2(x) # 一次前向传播（小批量）
        _, Pred_classes = torch.max(Pred, dim=1)  # 找到最大概率对应的类别索引
        correct_2 += torch.sum(Pred_classes == y)  # 计算正确的个数
        all_preds_2.extend(Pred_classes.cpu().numpy())  # 将预测转换为NumPy数组并存储
        all_labels_2.extend(y.cpu().numpy())  # 将真实标签转换为NumPy数组并存储
        total_2 += y.size(0)
accuracy_2 = correct_2/total_2

# 贝叶斯优化评价指标
accuracy_score_2 = accuracy_2
kappa_score_2 = cohen_kappa_score(all_labels_2, all_preds_2)
cm_2 = confusion_matrix(all_labels_2, all_preds_2)
cm_2_ = confusion_matrix(all_labels_2, all_preds_2, normalize='pred') # 计算列归一化的混淆矩阵
cm_2_normalized = (np.round(cm_2_ * 100)).astype(int) # 将归一化矩阵的元素乘以100并保留0位小数

print(f'贝叶斯优化的测试集精准度: {100*correct_2/total_2} %')

In [ ]:
report = classification_report(all_labels_2, all_preds_2,output_dict=True)
# 获取加权平均的精确率、召回率、F1系数
weighted_precision = report['macro avg']['precision']
weighted_recall = report['macro avg']['recall']
weighted_f1 = report['macro avg']['f1-score']

print(f"macro Precision: {weighted_precision}")
print(f"macro Recall: {weighted_recall}")
print(f"macro F1 Score: {weighted_f1}")